<a href="https://colab.research.google.com/github/prachigupta2006/feature-engineering/blob/main/column_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
from google.colab import files
uploaded = files.upload()

Saving covid_toy.csv to covid_toy.csv


In [4]:
df= pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


In [7]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

# WITHOUT COLUMN TRANSFORMATION

simple imputation

In [9]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

adding simple imputer to the fever column and also test the data

In [10]:
X_train_fever.shape

(80, 1)

ordinal encoding

In [15]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

OneHotEncoding

In [27]:
ohe=OneHotEncoder(drop='first',sparse=False)
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])

X_train_gender_city.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

we did one hot encoding on city and gender where we droped the first dummy column to avoid multicolinearity and thes we have gender = 1 and city = 3 that is 4 columns in total

EXTRACTING AGE

In [21]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [25]:
transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

transformed.shape

(80, 7)

# WITH COLUMN TRANSFORMER

In [28]:
from sklearn.compose import ColumnTransformer

creating the transformer


In [41]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Strong','Mild']],dtype=np.int32),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [45]:
transformer.fit_transform(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [104.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [103.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  16.        ],
       [102.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  82.        ],
       [102.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [100.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          0.    

In [46]:
transformer.fit_transform(X_train).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [47]:
transformer.fit_transform(X_test).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)